Práctica final Extracción, Transformación y Carga de datos
--

**La práctica se ha realizado mediante SQL.**

Antonio Nogués Podadera 

# Extracción 

Librerías

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect
import re 
import random
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

Conexión con la base de datos

In [2]:
# Objeto que representa la conexión a la base de datos

engine = create_engine('sqlite:///airbnb.sqlite') 

In [3]:
# Consulta para saber qué tablas tiene la base de datos

inspector = inspect(engine)
print(inspector.get_table_names())

['Hoods', 'Listings', 'Reviews']


## 1. Tabla `Listings`

In [4]:
query_1 = """
SELECT Listings.id, Hoods.neighbourhood_group, Listings.price, Listings.number_of_reviews, Listings.review_scores_rating, Listings.room_type
FROM Listings
JOIN Hoods 
ON Listings.neighbourhood_cleansed = Hoods.neighbourhood
"""

In [5]:
table_listings = pd.read_sql(query_1, engine)  
table_listings.head()

,id,neighbourhood_group,price,number_of_reviews,review_scores_rating,room_type
0,6369.0,Chamartín,$60.00,79.0,4.89,Private room
1,21853.0,Latina,$31.00,33.0,4.58,Private room
2,23001.0,Arganzuela,$50.00,NaN,NaN,Entire home/apt
3,24805.0,Centro,$92.00,12.0,4.92,Entire home/apt
4,26825.0,Arganzuela,$26.00,149.0,4.68,Private room


## 2. Tabla `Reviews`

In [6]:
query_2 = """
SELECT COUNT(Reviews.id) AS number_reviews, Hoods.neighbourhood_group, strftime('%Y-%m', Reviews.date) AS month
FROM Reviews
INNER JOIN Listings
ON Listings.id=Reviews.listing_id
INNER JOIN Hoods
ON Listings.neighbourhood_cleansed=Hoods.neighbourhood
WHERE strftime('%Y', Reviews.date) NOT LIKE 2010
GROUP BY Hoods.neighbourhood_group, month
"""

In [7]:
table_reviews = pd.read_sql(query_2, engine)  
table_reviews.head()

,number_reviews,neighbourhood_group,month
0,1,Arganzuela,2011-01
1,1,Arganzuela,2011-03
2,1,Arganzuela,2011-04
3,1,Arganzuela,2011-05
4,1,Arganzuela,2011-06


# Transformacion

## Transformación tabla `Listings`

### 3. Transformación de la columna `price`

Queremos eliminar el simbolo $ y la coma de los elementos de la columna "price"

In [8]:
# Selección de la columna en la que quiero aplicar la expresión regular (price)

col_price = table_listings['price']

# Aplicamos la expresión regular para eliminar el símbolo $

col_price = col_price.apply(lambda x: re.sub(r'\$', '', x))

# Aplicamos la expresión regular para eliminar la coma

col_price = col_price.apply(lambda x: re.sub(r',', '', x))

# Guardamos los cambios en el dataframe
table_listings['price'] = col_price

table_listings.head()

,id,neighbourhood_group,price,number_of_reviews,review_scores_rating,room_type
0,6369.0,Chamartín,60.00,79.0,4.89,Private room
1,21853.0,Latina,31.00,33.0,4.58,Private room
2,23001.0,Arganzuela,50.00,NaN,NaN,Entire home/apt
3,24805.0,Centro,92.00,12.0,4.92,Entire home/apt
4,26825.0,Arganzuela,26.00,149.0,4.68,Private room


Ahora podemos convertir la columna `price` a tipo numérico (float)

In [9]:
table_listings['price']= pd.to_numeric(table_listings['price'])

# Comprobación de que se ha convertido correctamente
table_listings['price'].dtype

dtype('float64')

### 4. Imputación de valores missing de `number_of_reviews` y `review_scores_rating`

Imputación de valores missing sobre `number_of_reviews`

In [10]:
# Agrupamos los datos por la columna 'room_type'
groups = table_listings.groupby('room_type')

In [11]:
# Semilla para que los resultados sean reproducibles
random.seed(12345)

# Iteración sobre cada grupo
for name, group in groups:
  # Seleccionamos los valores de la columna 'number_of_reviews' que no son NA
  values = group['number_of_reviews'].dropna().values

  # Iteración sobre cada fila del grupo
  for index, row in group.iterrows():
    # Si el valor de la columna 'number_of_reviews' es NA, rellenamos con un valor aleatorio
    if pd.isnull(row['number_of_reviews']):
      table_listings.at[index, 'number_of_reviews'] = random.choice(values)

Imputación de valores missing sobre `review_scores_rating`

In [12]:
# Semilla para que los resultados sean reproducibles
random.seed(12345)

# Iteración sobre cada grupo
for name, group in groups:
  # Seleccionamos los valores de la columna 'review_scores_rating' que no son NA
  values = group['review_scores_rating'].dropna().values

  # Iteración sobre cada fila del grupo
  for index, row in group.iterrows():
    # Si el valor de la columna 'review_scores_rating' es NA, rellenamos con un valor aleatorio
    if pd.isnull(row['review_scores_rating']):
      table_listings.at[index, 'review_scores_rating'] = random.choice(values)

In [13]:
# Se muestran los resultados de las dos columnas transformadas

table_listings.head()

,id,neighbourhood_group,price,number_of_reviews,review_scores_rating,room_type
0,6369.0,Chamartín,60.0,79.0,4.89,Private room
1,21853.0,Latina,31.0,33.0,4.58,Private room
2,23001.0,Arganzuela,50.0,8.0,4.88,Entire home/apt
3,24805.0,Centro,92.0,12.0,4.92,Entire home/apt
4,26825.0,Arganzuela,26.0,149.0,4.68,Private room


### 5. Agregación de datos

In [14]:
# Agrupamos el dataframe por distrito y tipo de alojamiento

group_df = table_listings.groupby(['neighbourhood_group', 'room_type'])

In [15]:
# Esta función calcula la media ponderada (review_scores_rating ponderado con number_of_reviews)

def calculo_media_ponderada(group):
  media_ponderada = (group["review_scores_rating"] * group["number_of_reviews"]).sum() / group["number_of_reviews"].sum()
  return media_ponderada

In [16]:
# Cálculo de la media ponderada

mean = group_df.apply(calculo_media_ponderada)
mean = pd.DataFrame(mean).reset_index().rename(columns={0: 'nota_media'})

In [17]:
# Cálculo del precio mediano

median_price = group_df['price'].median()
median_price = pd.DataFrame(median_price).reset_index().rename(columns={'price': 'precio_mediano'})

In [18]:
# Cálculo del número de alojamientos

count_id = group_df['id'].count()
count_id = pd.DataFrame(count_id).reset_index()

In [19]:
# Unión de dataframes

new_listings = pd.merge(count_id, median_price, on=['neighbourhood_group', 'room_type'])
new_listings = pd.merge(median_price, mean, on=['neighbourhood_group', 'room_type'])

new_listings.head()

,neighbourhood_group,room_type,precio_mediano,nota_media
0,Arganzuela,Entire home/apt,70.0,4.680992
1,Arganzuela,Hotel room,19.0,4.689091
2,Arganzuela,Private room,31.0,4.712139
3,Arganzuela,Shared room,44.0,4.186000
4,Barajas,Entire home/apt,86.0,4.713195


## Transformación tabla `Reviews`

### 6. Predicciones 

In [20]:
# Últimos valores de cada distrito

last_value = table_reviews.groupby('neighbourhood_group').tail(1)

In [21]:
# Se incrementa el mes de julio a agosto

last_value['month'] = last_value['month'].str.replace(r'2021-07', '2021-08')
last_value

,number_reviews,neighbourhood_group,month
116,67,Arganzuela,2021-08
216,19,Barajas,2021-08
322,24,Carabanchel,2021-08
449,888,Centro,2021-08
563,25,Chamartín,2021-08
683,60,Chamberí,2021-08
777,23,Ciudad Lineal,2021-08
871,19,Fuencarral - El Pardo,2021-08
982,27,Hortaleza,2021-08
1090,23,Latina,2021-08


In [22]:
# Unión de dataframes

new_reviews = pd.concat([table_reviews, last_value]).sort_values(by=['neighbourhood_group', 'month'])
new_reviews.head(5)

,number_reviews,neighbourhood_group,month
0,1,Arganzuela,2011-01
1,1,Arganzuela,2011-03
2,1,Arganzuela,2011-04
3,1,Arganzuela,2011-05
4,1,Arganzuela,2011-06


### 7. Casos en los que no hay datos

In [23]:
# Creación de todas las fechas posibles entre 2011-01 y 2021-08

month = pd.date_range('2011-01', '2021-08', freq='MS')

In [24]:
# Posibles distritos

neighbourhood_group = pd.Series(last_value['neighbourhood_group'].unique())

In [25]:
# Unión de los dataframes de fechas y distritos creados anteriormente. 

combinations = pd.MultiIndex.from_product([month, neighbourhood_group], names=['month', 'neighbourhood_group']).to_frame().reset_index(drop=True)
combinations.head()

,month,neighbourhood_group
0,2011-01-01,Arganzuela
1,2011-01-01,Barajas
2,2011-01-01,Carabanchel
3,2011-01-01,Centro
4,2011-01-01,Chamartín


In [26]:
# Modificación del formato de la columna 'month' a año y mes 

combinations['month'] = combinations['month'].apply(lambda x: datetime.strftime(x, '%Y-%m'))
combinations.head()

,month,neighbourhood_group
0,2011-01,Arganzuela
1,2011-01,Barajas
2,2011-01,Carabanchel
3,2011-01,Centro
4,2011-01,Chamartín


In [27]:
# Unión de dataframes
new_reviews = pd.merge(combinations, new_reviews, on=['month', 'neighbourhood_group'], how='outer')

# Se ordenan los valores por distrito y mes
new_reviews = new_reviews.sort_values(by=['neighbourhood_group', 'month'])

new_reviews.head()

,month,neighbourhood_group,number_reviews
0,2011-01,Arganzuela,1.0
21,2011-02,Arganzuela,NaN
42,2011-03,Arganzuela,1.0
63,2011-04,Arganzuela,1.0
84,2011-05,Arganzuela,1.0


Tenemos valores nulos en la columna number_reviews puesto que en esa fecha nueva que se ha creado no hay ningún review. Para solucionarlo se imputara un 0 a estos valores. 

In [28]:
new_reviews['number_reviews'] = new_reviews['number_reviews'].fillna(0)
new_reviews.head()

,month,neighbourhood_group,number_reviews
0,2011-01,Arganzuela,1.0
21,2011-02,Arganzuela,0.0
42,2011-03,Arganzuela,1.0
63,2011-04,Arganzuela,1.0
84,2011-05,Arganzuela,1.0


# 8. Carga 

## Carga del dataframe `new_listings` a la base de datos

In [29]:
# Carga del dataframe en la base de datos. 
# El argumento "if_exists='replace'" se ha añadido para que al volver a ejecutar el código no de error al intentar crear una tabla que ya existe

new_listings.to_sql('new_listings', engine, if_exists='replace', index=True)

75

Comprobación de la carga: **Consulta de los datos de la tabla "new_listings"**

In [30]:
check_listings_query = """
SELECT * 
FROM new_listings LIMIT 10
"""

In [31]:
table_reviews = pd.read_sql(check_listings_query, engine)  
table_reviews.head()

,index,neighbourhood_group,room_type,precio_mediano,nota_media
0,0,Arganzuela,Entire home/apt,70.0,4.680992
1,1,Arganzuela,Hotel room,19.0,4.689091
2,2,Arganzuela,Private room,31.0,4.712139
3,3,Arganzuela,Shared room,44.0,4.186000
4,4,Barajas,Entire home/apt,86.0,4.713195


## Carga del dataframe `new_reviews` a la base de datos

In [32]:
# Carga del dataframe en la base de datos
# El argumento "if_exists='replace'" se ha añadido para que al volver a ejecutar el código no de error al intentar crear una tabla que ya existe.

new_reviews.to_sql('new_reviews', engine, if_exists='replace', index=True)

2688

Comprobación de la carga: **Consulta de los datos de la tabla "new_reviews"**

In [33]:
check_reviews_query = """
SELECT * 
FROM new_reviews LIMIT 10
"""

In [34]:
table_reviews = pd.read_sql(check_reviews_query, engine)  
table_reviews.head()

,index,month,neighbourhood_group,number_reviews
0,0,2011-01,Arganzuela,1.0
1,21,2011-02,Arganzuela,0.0
2,42,2011-03,Arganzuela,1.0
3,63,2011-04,Arganzuela,1.0
4,84,2011-05,Arganzuela,1.0
